# Airbnb project
## neighbourhood analysis
How does neighbourhood affect the price and ratings?

## Import relevant pacakges

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline

## Data gathering

In [15]:
df_listings = pd.read_csv('data/listings.csv.gz')

C:\Users\yu298\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data assessing

In [16]:
df_listings.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24678 entries, 0 to 24677
Data columns (total 106 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            24678 non-null  int64  
 1   listing_url                                   24678 non-null  object 
 2   scrape_id                                     24678 non-null  int64  
 3   last_scraped                                  24678 non-null  object 
 4   name                                          24628 non-null  object 
 5   summary                                       23447 non-null  object 
 6   space                                         15953 non-null  object 
 7   description                                   24160 non-null  object 
 8   experiences_offered                           24678 non-null  object 
 9   neighborhood_overview                         13851 non-null

Extract columns of interest, namely
- id
- neighbourhood_group_cleansed
- price
- review_scores_rating

In [17]:
df_neibourhood = df_listings[['id', 'neighbourhood_group_cleansed', 'price', 'review_scores_rating']]
df_neibourhood

,id,neighbourhood_group_cleansed,price,review_scores_rating
0,2015,Mitte,$60.00,93.0
1,3176,Pankow,$90.00,93.0
2,3309,Tempelhof - Schöneberg,$28.00,89.0
3,7071,Pankow,$33.00,97.0
4,9991,Pankow,$180.00,100.0
...,...,...,...,...
24673,43422635,Friedrichshain-Kreuzberg,$65.00,NaN
24674,43430815,Spandau,$35.00,NaN
24675,43432267,Treptow - Köpenick,$40.00,NaN
24676,43436282,Treptow - Köpenick,$46.00,NaN


In [18]:
df_neibourhood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24678 entries, 0 to 24677
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            24678 non-null  int64  
 1   neighbourhood_group_cleansed  24678 non-null  object 
 2   price                         24678 non-null  object 
 3   review_scores_rating          19645 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 771.3+ KB


Few quality issues with data
- price has non-numeric data type
- about 5000 data missing from review scores

In [19]:
# convert price column.  Remove ',' and '$' sign, then change datatype.
def price_conversion(df,col):
    df.loc[:,col] = df.loc[:,col].str.replace(',','')
    df.loc[:,col] = pd.to_numeric(df.loc[:,col].str.replace('$', ''))

In [20]:
price_conversion(df_neibourhood, 'price')

C:\Users\yu298\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [21]:
df_neibourhood.describe()

,id,price,review_scores_rating
count,2.467800e+04,24678.000000,19645.000000
mean,2.199784e+07,75.109409,94.583049
std,1.262602e+07,239.989351,7.876218
min,2.015000e+03,0.000000,20.000000
25%,1.122274e+07,35.000000,92.000000
50%,2.144830e+07,50.000000,97.000000
75%,3.296801e+07,79.000000,100.000000
max,4.343876e+07,9000.000000,100.000000


In order to deal with the missing values, lets first see if any particular region has higher/lower percentage of missing review.